# Napredne funkcionalnosti Pandas

V tej enoti se boste seznanili z nekaj naprednimi funkcionalnostmi Pandas in dobrimi praksami pri obdelavi velepodatkov. Tematike te enote bodo optimizacija podatkovnih tipov, veriženje funkcij in nekatere napredne funkcije knjižnice Pandas. Pri delu bomo uporabljali lastno funkcijo za generiranje poljubno velike zbirke velepodatkov. Ustvarjene velepodatke bomo uporabljali pri analizi in optimizaciji podatkovnih tipov ter kot izhodišče za izvedbo veriženih funkcij. Nad njimi bomo zaganjali tudi nekaj naprednih funkcij.

## Ustvarjanje zbirke velepodatkov

Najprej si bomo pripravili lastno funkcijo za ustvarjanje poljubno velike zbirke velepodatkov. Ustvarili bomo velepodatke z opisi igralcev v ekipah. Igralec bo imel naslednje lastnosti:

* Položaj (levo, sredina, desno)
* Starost (vrednost med 18 in 50)
* Ekipa (rdeča, modra, rumena, zelena)
* Zmaga (da, ne)
* Verjetnost (vrednost med 0 in 1)

Pripravimo funkcijo za ustvarjanje velepodatkov:

In [36]:
import pandas as pd
import numpy as np

In [37]:
def generate_data(size):
    df = pd.DataFrame()
    df['Polozaj'] = np.random.choice(['levo', 'sredina', 'desno'], size)
    df['Starost'] = np.random.randint(18, 50, size)
    df['Ekipa'] = np.random.choice(['rdeča', 'modra', 'rumena', 'zelena'], size)
    df['Zmaga'] = np.random.choice(['da', 'ne'], size)
    df['Verjetnost'] = np.random.uniform(0, 1, size)
    return df

Zaženimo funkcijo in ustvarimo zbirko velepodatkov z 10 milijoni vrstic. S funkcijo ``df.info()`` bomo videli podrobnosti naše zbirke, s funkcijo ``df.head()`` pa bomo izpisali prvih 5 vrstic.

In [38]:
df = generate_data(10_000_000)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Polozaj     object 
 1   Starost     int32  
 2   Ekipa       object 
 3   Zmaga       object 
 4   Verjetnost  float64
dtypes: float64(1), int32(1), object(3)
memory usage: 343.3+ MB


,Polozaj,Starost,Ekipa,Zmaga,Verjetnost
0,sredina,34,rdeča,da,0.728796
1,sredina,37,modra,ne,0.073443
2,levo,38,modra,ne,0.367767
3,sredina,49,zelena,ne,0.271946
4,desno,30,rdeča,da,0.375633


Izvedimo nekaj agregatnih funkcij (``.groupby()`` in ``.rank()``) nad ustvarjeno zbirko velepodatkov in ob tem merimo čas izvedbe. Čas bomo merili z ukazom ``%timeit``, ki meri čas izvedbe ene vrstice v celici Jupyter zvezka. Ta ukaz vrstico izvede večkrat in vrne povprečen čas izvedbe s standardnim odklonom. 

In [39]:
# naredimo kopijo velepodatkov
df_result = df.copy()

# Ustvarimo nov stolpec z imenom 'Starost_Rang', ki bo vseboval številčno vrednost ranga, če izvedemo združevanje najprej po ekipi, nato pa po položaju. 
%timeit df_result['Starost_Rang'] = df_result.groupby(['Ekipa', 'Polozaj'], observed=True)['Starost'].rank()

# Ustvarimo nov stolpec z imenom 'Verjetnost_Rang', ki bo vseboval številčno vrednost ranga, če izvedemo združevanje najprej po ekipi, nato pa po položaju.
%timeit df_result['Verjetnost_Rang'] = df_result.groupby(['Ekipa', 'Polozaj'], observed=True)['Verjetnost'].rank()

# Ustvarimo nov stolpec z imenom 'VerjetnostZmage_Rang', ki bo vseboval številčno vrednost ranga, če izvedemo združevanje najprej po ekipi, nato po položaju, zatem pa še po zmagi.
%timeit df_result['VerjetnostZmage_Rang'] = df_result.groupby(['Ekipa', 'Polozaj', 'Zmaga'], observed=True)['Verjetnost'].rank()

7.92 s ± 96.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
9.46 s ± 63.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
10.6 s ± 403 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Izvedli smo verižene agregatne funkcije in izmerili povprečen čas izvedbe:

* Prva vrstica: 7,92s
* Druga vrstica: 9,46s
* Tretja vrstica: 10,6s

Poglejmo, ali lahko pohitrimo izvedbo teh agregatnih funkcij.

## Optimizacija podatkovnih tipov

Poskusimo optimizirati podatkovne tipe, ki jih uporablja naša zbirka velepodatkov. Najprej poglejmo obstoječe podatkovne tipe, nato pa razmislimo kako bi jih lahko optimizirali. Obstoječe podatkovne tipe lahko preverimo s funkcijo ``.info()``. 

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Polozaj     object 
 1   Starost     int32  
 2   Ekipa       object 
 3   Zmaga       object 
 4   Verjetnost  float64
dtypes: float64(1), int32(1), object(3)
memory usage: 343.3+ MB


Obstoječe podatkovne tipe lahko optimiziramo. Stolpca ``Polozaj`` in ``Ekipa`` sta kategoričnega tipa in ju lahko ustrezno pretvorimo. Nadalje, številčna stolpca ``Starost`` in ``Verjetnost`` lahko spremenimo v bolj primerna številčna podatkovnega tipa. Stolpec ``Zmaga`` opisuje binarno stanje in ga lahko prav tako pretvorimo. 

Poglejmo še porabo pomnilnika - z obstoječimi podatkovnimi tipi je poraba pomnilnika okoli 300MB. Pričakujemo lahko, da bomo z optimizacijo podatkovnih tipov zmanjšali tudi porabo pomnilnika. Poglejmo si torej postopek optimizacije podatkovnih tipov.

In [41]:
# optimizacija podatkovnih tipov

df_optimized = df.copy() # naredimo kopijo podatkov

# pretvorba stolpca 'Polozaj' v kategorični tip
# str => category
df_optimized['Polozaj'] = df_optimized['Polozaj'].astype('category')

# pretvorba stolpca 'Ekipa' v kategorični tip
# str => category
df_optimized['Ekipa'] = df_optimized['Ekipa'].astype('category')

# pretvorba stolpca 'Starost' v 8-bitno celo število (brez predznaka) [0, 255]
# int64 => uint8
df_optimized['Starost'] = df_optimized['Starost'].astype('uint8')

# pretvorba stolpca 'Verjetnost' v 32-bitno realno število enojne natančnosti
# float64 => float32
df_optimized['Verjetnost'] = df_optimized['Verjetnost'].astype('float32')

# pretvorba stolpca 'Zmaga' v binarni tip
# str => bool
df_optimized['Zmaga'].map({'da': True, 'ne': False})

# izpis optimiziranih podatkovnih tipov
df_optimized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 5 columns):
 #   Column      Dtype   
---  ------      -----   
 0   Polozaj     category
 1   Starost     uint8   
 2   Ekipa       category
 3   Zmaga       object  
 4   Verjetnost  float32 
dtypes: category(2), float32(1), object(1), uint8(1)
memory usage: 143.1+ MB


Vidimo, da smo spremenili podatkovne tipe, s tem pa smo zmanjšali porabo pomnilnika za približno polovico (poraba pomnilnika je zdaj okoli 140MB). Poglejmo še, ali optimizacija podatkovnih tipov vpliva na hitrost agregatnih funkcij. 

Ponovno bomo izvedli enake agregatne funkcije kot prej in izmerili povprečen čas izvedbe.

In [42]:
# Ustvarimo nov stolpec z imenom 'Starost_Rang', ki bo vseboval številčno vrednost ranga, če izvedemo združevanje najprej po ekipi, nato pa po položaju. 
%timeit df_optimized['Starost_Rang'] = df_optimized.groupby(['Ekipa', 'Polozaj'], observed=True)['Starost'].rank()

# Ustvarimo nov stolpec z imenom 'Verjetnost_Rang', ki bo vseboval številčno vrednost ranga, če izvedemo združevanje najprej po ekipi, nato pa po položaju.
%timeit df_optimized['Verjetnost_Rang'] = df_optimized.groupby(['Ekipa', 'Polozaj'], observed=True)['Verjetnost'].rank()

# Ustvarimo nov stolpec z imenom 'VerjetnostZmage_Rang', ki bo vseboval številčno vrednost ranga, če izvedemo združevanje najprej po ekipi, nato po položaju, zatem pa še po zmagi.
%timeit df_optimized['VerjetnostZmage_Rang'] = df_optimized.groupby(['Ekipa', 'Polozaj', 'Zmaga'], observed=True)['Verjetnost'].rank()

6.62 s ± 154 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
8.68 s ± 69.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
9.22 s ± 74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Z optimiziranimi podatkovnimi tipi smo dobili naslednje povprečne čase izvedbe:

* Prva vrstica: 6,62s (prej 7,92s)
* Druga vrstica: 8,68s (prej 9,46s)
* Tretja vrstica: 9,22s (prej 10,6s)

Vidimo torej, da se optimizacija podatkovnih tipov splača, saj smo v vseh treh primerih dosegli približno pohitritev za 1 sekundo.

## Veriženje funkcij

Veriženje funkcij *(ang. method chaining)* je zmožnost zaporedne izvedbe funkcij Pandas. Z veriženjem lahko enostavno opišemo zaporedje izvedenih funkcij prav tako pa je takšna koda lažja za razumevanje, vzdrževanje in nam olajša iskanje napak. Gre za dobro prakso, ki se pogosto uporablja pri napredni obdelavi podatkov in velepodatkov.

Poglejmo si primer brez in z veriženjem funkcij:

In [52]:
# najprej naredimo kopijo podatkov
df_tmp = df_optimized.copy()

In [65]:
%%timeit 
# brez veriženja izberimo vse igralce, ki so stari več kot 30 let in igrajo za rdečo ekipo, nato pa jih uredimo po starosti naraščujoče in izberimo prvih 10

df_nonchain = df_tmp.where(df_tmp['Starost'] >= 30)
df_nonchain = df_tmp.where(df_tmp['Ekipa'] == 'rdeča')
df_nonchain = df_tmp.dropna()
df_nonchain = df_tmp.sort_values('Starost', ascending=True)
df_nonchain.head(10)


2.25 s ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
%%timeit
# ponovimo isto z veriženjem
df_tmp.where(df_tmp['Starost'] >= 30).where(df_tmp['Ekipa'] == 'rdeča').dropna().sort_values('Starost', ascending=True).head(10)

1.18 s ± 9.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Z ukazom ``%%timeit`` smo merili čas izvedbe obeh pristopov. Vidimo, da je pristop z veriženjem v povprečju dvakrat hitrejši od pristopa brez veriženja. To je predvsem zaradi shranjevanja vmesnega rezultata pri pristopu brez veriženja. Kadar to ni potrebno za končni rezultat, lahko takšno shranjevanje izpustimo. 

Opazimo, da je pri pristopu z veriženjem dvakrat uporabljena funkcija ``.where()``. Poskusimo združiti pogoja v obeh primerih uporabe ``.where()`` in poglejmo ali to vpliva na hitrost izvedbe. 

In [68]:
%%timeit
# ponovimo isto z veriženjem, tokrat združujemo pogoja v .where()
# uporabili smo operator &, ki predstavlja logično operacijo 'in'
# oba pogoja smo obdali z oklepaji
df_tmp.where((df_tmp['Starost'] >= 30) & (df_tmp['Ekipa'] == 'rdeča')).dropna().sort_values('Starost', ascending=True).head(10)

858 ms ± 5.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Z združevanjem pogojev v ``.where()`` smo dosegli še dodatno pohitritev. Bistveno je razumeti, da za hitro obdelavo velepodatkov uporabimo vse možne optimizacije, ki so nam na voljo. 

Veriženje funkcij je očitno uporabno, vendar je s takšnim pristopom zapis programske kode v eni vrstici lahko nepregleden. To vodi v slabo berljivost programske kode in posledično oteženo reševanje napak. Zapis lahko naredimo bolj pregleden z uporabo oklepajev in tabulatorjev, kot je prikazano v spodnjem primeru.

In [69]:
%%timeit
# izboljšan zapis pristopa z veriženjem

(df_tmp
    .where((df_tmp['Starost'] >= 30) & (df_tmp['Ekipa'] == 'rdeča'))
    .dropna()
    .sort_values('Starost', ascending=True)
    .head(10)
)

850 ms ± 4.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Napredne funkcije

Knjižnica Pandas podpira različne napredne funkcije za delo s podatki. Teh je veliko in jih lahko podrobneje preučite s pomočjo dokumentacije:

* [``.apply()``](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html#pandas.Series.apply)
* [``.nunique()``](https://pandas.pydata.org/docs/reference/api/pandas.Series.nunique.html#pandas.Series.nunique)
* [``.sort_values()``](https://pandas.pydata.org/docs/reference/api/pandas.Series.sort_values.html#pandas.Series.sort_values)
* [``.rename()``](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html#pandas.DataFrame.rename)
* [``.groupby()``](https://pandas.pydata.org/docs/reference/groupby.html#groupby)
* [``.query()``](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html#pandas.DataFrame.query)
* [``.melt()``](https://pandas.pydata.org/docs/reference/api/pandas.melt.html#pandas.melt)
* [``.crosstab()``](https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html#pandas-crosstab)
* [``.pivot_table()``](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html#pandas-pivot-table)
* [``.cut()``](https://pandas.pydata.org/docs/reference/api/pandas.cut.html#pandas.cut)
* [``.isin()``](https://pandas.pydata.org/docs/reference/api/pandas.Series.isin.html#pandas.Series.isin)
* [``.value_counts()``](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html#pandas.Series.value_counts)
* [``.rolling()``](https://pandas.pydata.org/docs/reference/api/pandas.Series.rolling.html#pandas.Series.rolling)

V nadaljevanju si bomo podrobneje pogledali funkciji ``.apply()`` in ``.query()``, ki sta lahko zelo uporabni pri obdelavi podatkovnih zbirk.

### ``.apply()``

Funkcija ``.apply()`` omogoča izvedbo poljubne funkcije nad podatki. To so lahko vgrajene funkcije ali pa tudi naše lastne funkcije. To je uporabno v primerih, kadar želimo obdelati vse podatke v enem stolpcu. Prvi pristop, ki bi ga najverjetneje uporabili je implementacija zanke, ki spreminja posamezni podatek v stolpcu. To se izkaže za izredno časovno potratno, zato je bolj smiselno uporabiti funkcije, ki paketno obdelujejo podatke. 

Poglejmo si primer funkcije ``.apply()`` z vgrajeno funkcijo. Sami lahko poskusite implementirati enako operacijo s pomočjo zanke in primerjate izvajalni čas.

In [ ]:
# V tej celici poskusite sami implementirati operacijo .apply() s pomočjo zanke. Primerjajte hitrost izvajanja z vgrajeno funkcijo .apply().

In [79]:
# najprej naredimo kopijo podatkov brez določenih stolpcev
df_app1 = df_tmp.copy().drop(['Starost_Rang', 'Verjetnost_Rang', 'VerjetnostZmage_Rang'], axis=1)

# uporabimo funkcijo apply() za izračun kvadratnega korena vrednosti stolpca 'Starost'
# pri tem uporabimo numpy funkcijo np.sqrt()
df_app1['Koren_Starost'] = df_tmp['Starost'].apply(np.sqrt)

df_app1.head()

,Polozaj,Starost,Ekipa,Zmaga,Verjetnost,Koren_Starost
0,sredina,34,rdeča,da,0.728796,5.832031
1,sredina,37,modra,ne,0.073443,6.082031
2,levo,38,modra,ne,0.367767,6.164062
3,sredina,49,zelena,ne,0.271946,7.000000
4,desno,30,rdeča,da,0.375633,5.476562


Uporabimo še funkcijo ``.apply()`` z lastno funkcijo. Za vsakega igralca bomo na podlagi starosti uvrstili v eno izmed kategorij. Kategorije bodo "U21", "U23", "adult" in "veteran". 

In [80]:
# lastna funkcija za katogorizacijo igralcev glede na starost
def categorize(age):
    category = None
    if(age <= 21): category = 'U21'
    if(age > 21 and age <= 23): category = 'U23'
    if(age > 23 and age <= 34): category = 'adult'
    if(age > 34): category = 'veteran'

    return category

# naredimo kopijo podatkov brez določenih stolpcev
df_app2 = df_tmp.copy().drop(['Starost_Rang', 'Verjetnost_Rang', 'VerjetnostZmage_Rang'], axis=1)

# uporabimo funkcijo .apply() za kategorizacijo igralcev
df_app2['Kategorija'] = df_app2['Starost'].apply(categorize)

df_app2.head()

,Polozaj,Starost,Ekipa,Zmaga,Verjetnost,Kategorija
0,sredina,34,rdeča,da,0.728796,adult
1,sredina,37,modra,ne,0.073443,veteran
2,levo,38,modra,ne,0.367767,veteran
3,sredina,49,zelena,ne,0.271946,veteran
4,desno,30,rdeča,da,0.375633,adult


Nazadnje si poglejmo še uporabo funkcije ``.apply()`` z [anonimno oz. **lambda** funkcijo](https://www.geeksforgeeks.org/python-lambda-anonymous-functions-filter-map-reduce/). Vse ekipe zapišimo z velikimi tiskanimi črkami.

In [81]:
# naredimo kopijo podatkov brez določenih stolpcev
df_app3 = df_tmp.copy().drop(['Starost_Rang', 'Verjetnost_Rang', 'VerjetnostZmage_Rang'], axis=1)

# uporabimo funkcijo .apply() za pretvorbo v velike črke z anonimno (lambda) funkcijo
df_app3['Ekipa'] = df_app3['Ekipa'].apply(lambda x: x.upper())

df_app3.head()


,Polozaj,Starost,Ekipa,Zmaga,Verjetnost
0,sredina,34,RDEČA,da,0.728796
1,sredina,37,MODRA,ne,0.073443
2,levo,38,MODRA,ne,0.367767
3,sredina,49,ZELENA,ne,0.271946
4,desno,30,RDEČA,da,0.375633


### ``.query()``



Funkcija ``.query()`` je ena izmed zelo zmogljivih funkcij knjižnice Pandas, ki nam omogoča poizvedovanje. Z ustrezno poizvedbo lahko na hiter in berljiv način izberemo tiste podatke, ki nas v naši podatkovni zbirki zanimajo. Uporaba te funkcije je zelo podobna uporabi poizvedbam SQL *(ang. Structured Query Language)* v podatkovnih bazah.

Poglejmo si primere funkcije ``.query()`` na preprostih in kompleksnih poizvedbah.

In [86]:
# naredimo kopijo podatkov brez določenih stolpcev
df_q = df_tmp.copy().drop(['Starost_Rang', 'Verjetnost_Rang', 'VerjetnostZmage_Rang'], axis=1)

# uporabimo funkcijo .query() za izbor igralcev, ki so stari več kot 25 let in izberimo prvih 5

# pogoj navedemo v enojnih narekovajih, stolpec pa znotraj posebnih narekovajev (``) 
df_q.query('`Starost` > 25').head(5)

,Polozaj,Starost,Ekipa,Zmaga,Verjetnost
0,sredina,34,rdeča,da,0.728796
1,sredina,37,modra,ne,0.073443
2,levo,38,modra,ne,0.367767
3,sredina,49,zelena,ne,0.271946
4,desno,30,rdeča,da,0.375633


In [90]:
# poskusimo še z več pogoji: igralci stari med 25 in 28, ki igrajo za rdečo ekipo in imajo vsaj 75% verjetnost zmage

# uporabimo logično operacijo 'in' (uporabimo lahko "and" ali "&")
df_q.query('`Starost` >= 25 and `Starost` <= 28 and `Ekipa` == "rdeča" and `Zmaga` == "da" and `Verjetnost` >= 0.75')

,Polozaj,Starost,Ekipa,Zmaga,Verjetnost
197,sredina,26,rdeča,da,0.953634
904,desno,25,rdeča,da,0.882663
1464,sredina,27,rdeča,da,0.855277
1519,sredina,25,rdeča,da,0.840098
1838,desno,25,rdeča,da,0.988162
...,...,...,...,...,...
9998484,desno,25,rdeča,da,0.813323
9998839,sredina,26,rdeča,da,0.952503
9999107,sredina,28,rdeča,da,0.875873
9999957,desno,27,rdeča,da,0.877774


In [91]:
# poglejmo še delovanje z veriženjem, kjer dobljene igralce uredimo padajoče po verjetnosti in izberemo prvih 5
(df_q
 .query('`Starost` >= 25 & `Starost` <= 28 & `Ekipa` == "rdeča" & `Zmaga` == "da" & `Verjetnost` >= 0.75')
 .sort_values('Verjetnost', ascending=False)
 .head(5)
)

,Polozaj,Starost,Ekipa,Zmaga,Verjetnost
638490,levo,26,rdeča,da,0.999997
2553477,sredina,27,rdeča,da,0.999987
6887193,levo,25,rdeča,da,0.999984
6000648,sredina,28,rdeča,da,0.999972
8052346,sredina,28,rdeča,da,0.999970
